<a href="https://colab.research.google.com/github/JulioHaro93/PracticasTesisKeras/blob/main/TensorFlow_Serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  2943  100  2943    0     0   3359      0 --:--:-- --:--:-- --:--:--  3359
OK


In [35]:
! apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package tensorflow-model-server


In [40]:
!pip install tensorflow-serving

ERROR: Could not find a version that satisfies the requirement tensorflow-serving (from versions: none)
ERROR: No matching distribution found for tensorflow-serving


In [42]:
docker run -p 8501:8501 \
  --mount type=bind,source=/path/a/saved_model,target=/models/my_model \
  -e MODEL_NAME=my_model \
  -t tensorflow/serving


SyntaxError: invalid syntax (<ipython-input-42-e64c53be1840>, line 1)

In [7]:
!pip install tensorflow

In [8]:
!pip install requests

In [9]:
import os
import json
import random
import requests
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
%matplotlib inline

tf.__version__

'2.19.0'

# **Paso 3: Pre procesado de los datos**

In [10]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [11]:
class_names = ['avión', 'coche', 'pájaro', ' gato', 'ciervo', 'perro', 'rana', 'caballo','barco', 'camión']

#normalización

In [12]:
X_train = X_train / 255.0
X_test = X_test / 255.0


In [13]:
X_train.shape #Recordar que es importante conocer los tamaños de las imágenes

(50000, 32, 32, 3)

# Paso 4: Definir el modelo

Mismo modelo de CNN

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, padding="same", activation="relu", input_shape=[32,32,3]))
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, padding="same", activation="relu"))
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding='valid'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])


In [16]:
model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - loss: 1.7250 - sparse_categorical_accuracy: 0.3782
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 1.0647 - sparse_categorical_accuracy: 0.6280
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.8791 - sparse_categorical_accuracy: 0.6932
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.7522 - sparse_categorical_accuracy: 0.7387
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.6581 - sparse_categorical_accuracy: 0.7726
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.5761 - sparse_categorical_accuracy: 0.7956
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4966 - sparse_categorical_accuracy: 0.8248
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.4215 - sparse_categorical_accuracy: 0.8524
Epoch 9/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.3501 - sparse_categorical_accuracy: 0.8790
Epoch 10/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 1

In [17]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.8932 - sparse_categorical_accuracy: 0.7486


In [18]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.7488999962806702


#**paso 5: Guardar modelo para subir a producción**

Creamos el directorio del modelo

In [19]:
MODEL_DIR = "model/"
version =1

export_path = os.path.join(MODEL_DIR, str(version))
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')

In [22]:
if os.path.isdir(export_path):
  print('\nAlready saved a model, cleaning up\n')
  !rm -r {export_path}

In [45]:
model.export(export_path)

Saved artifact at 'model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  137324922770192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137324922770960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137324922770384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322299031184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292756752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292758480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292757520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292757136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292759248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292760592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137322292760400: TensorS

# **Paso 6: Configurar el entorno de producción**

**Exportar el MODEL_DIR a las variables de entorno**

Se hace con el path absoluto

In [ ]:
os.environ['MODEL_DIR'] = os.path.abspath(MODEL_DIR)

**Ejecutar la api rest de tensorflow serving, se hace desde la terminal de linux**

In [46]:
%%bash --bg
nohup tensorflow_model_server --rest_api_port=8501 --model_name=cifar10 --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [50]:
!tail server.log

nohup: failed to run command 'tensorflow_model_server': No such file or directory


In [43]:
random_image = np.random.randint(0,len(X_test))
random_image

4997

In [44]:
data = json.dumps({"signature_name": "serving_default", "instances": X_test[random_image].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... 3137255, 0.4392156862745098, 0.37254901960784315]]]}


In [49]:
!headers ?{"content-type":"application/json"} -d

/bin/bash: line 1: headers: command not found


In [56]:
headers ={"content-type":"application/json"}
json_response = requests.post(url='http://localhost:8080/v1/models/cifar10:predict', data=data, headers=headers)

In [57]:
json_response

<Response [404]>

In [58]:
predictions = json.loads(json_response.text)['predictions']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [59]:
predictions

NameError: name 'predictions' is not defined